In [1]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import seaborn as sns

import bokeh.plotting as bpl
import bokeh.models as bmo
from bokeh.palettes import d3
from bokeh.io import output_file, show
from bokeh.models.widgets import Select
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.layouts import column
from bokeh.models.widgets import RangeSlider
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn
bpl.output_notebook()

Loading BokehJS ...

In [2]:
real_estate_readings = pd.read_csv("Sacramentorealestatetransactions.csv")
real_estate_readings.head()

,street,city,zip,state,beds,baths,sq__ft,type,sale_date,price,latitude,longitude
0,3526 HIGH ST,SACRAMENTO,95838,CA,2,1,836,Residential,Wed May 21 00:00:00 EDT 2008,59222,38.631913,-121.434879
1,51 OMAHA CT,SACRAMENTO,95823,CA,3,1,1167,Residential,Wed May 21 00:00:00 EDT 2008,68212,38.478902,-121.431028
2,2796 BRANCH ST,SACRAMENTO,95815,CA,2,1,796,Residential,Wed May 21 00:00:00 EDT 2008,68880,38.618305,-121.443839
3,2805 JANETTE WAY,SACRAMENTO,95815,CA,2,1,852,Residential,Wed May 21 00:00:00 EDT 2008,69307,38.616835,-121.439146
4,6001 MCMAHON DR,SACRAMENTO,95824,CA,2,1,797,Residential,Wed May 21 00:00:00 EDT 2008,81900,38.519470,-121.435768


In [3]:
def modify_doc(doc):
    
    # Create working copy of data frame
    df = real_estate_readings.copy()
    source = bpl.ColumnDataSource(df)

    # create mouseover effects
    TOOLTIPS = [
        ("address", "@street"),
        ("price", "@price"),
        ("sq ft", "@sq__ft"),
        ("beds", "@beds"),
        ("baths", "@baths"),
    ]

    # create colors
    palette = d3['Category10'][len(df['type'].unique())]
    color_map = bmo.CategoricalColorMapper(factors=df['type'].unique(),
                                       palette=palette)

    # create figure and plot
    plot = bpl.figure(plot_width = 800, plot_height = 800, 
               title = 'interactive real estate plot',
               x_axis_label = 'latitude', y_axis_label = 'longitude', tooltips=TOOLTIPS)
    plot.scatter(x='latitude', y='longitude',
              color={'field': 'type', 'transform': color_map}, source=source)

    # Callback method for adjusting sliders
    def callback(attr, old, new):
        price_start = price_select.value[0]
        price_end = price_select.value[1]
        sq_ft_start = sq_ft_select.value[0]
        sq_ft_end = sq_ft_select.value[1]
        beds_start = beds_select.value[0]
        beds_end = beds_select.value[1]
        baths_start = baths_select.value[0]
        baths_end = baths_select.value[1]
        data = df
        data = data.loc[data['price'] >= price_start]
        data = data.loc[data['price'] < price_end]
        data = data.loc[data['sq__ft'] >= sq_ft_start]
        data = data.loc[data['sq__ft'] < sq_ft_end]
        data = data.loc[data['beds'] >= beds_start]
        data = data.loc[data['beds'] < beds_end]
        data = data.loc[data['baths'] >= baths_start]
        data = data.loc[data['baths'] < baths_end]
        source.data = ColumnDataSource(data=data).data

    # Create Sliders
    price_select = RangeSlider(start = 0, end = 1000000, 
                         step = 1000, value=(0,1000000),
                         title = 'price', width=900)
    sq_ft_select = RangeSlider(start = 0, end = 10000, 
                         step = 100, value = (0,10000),
                         title = 'sq ft', width=900)
    beds_select = RangeSlider(start = 0, end = 10, 
                         step = 1, value = (0,10),
                         title = 'beds', width=900)
    baths_select = RangeSlider(start = 0, end = 7, 
                         step = 1, value = (0,7),
                         title = 'baths', width=900)

    # Add functionality to sliders
    price_select.on_change('value', callback)
    sq_ft_select.on_change('value', callback)
    beds_select.on_change('value', callback)
    baths_select.on_change('value', callback)
    
    # Create DataTable to display currently showing results
    columns = [
        TableColumn(field="street", title="Street"),
        TableColumn(field="city", title="City"),
        TableColumn(field="zip", title="Zip"),
        TableColumn(field="state", title="State"),
        TableColumn(field="beds", title="Beds"),
        TableColumn(field="baths", title="Baths"),
        TableColumn(field="sq__ft", title="Square Feet"),
        TableColumn(field="type", title="Type"),
        TableColumn(field="sale_date", title="Sale Date"),
        TableColumn(field="price", title="Price"),
        TableColumn(field="latitude", title="Latitude"),
        TableColumn(field="longitude", title="Longitude")
    ]
    data_table = DataTable(source=source, columns=columns, width=900, height=400)
    
    # Add all displayable components to root
    doc.add_root(column(price_select, sq_ft_select, beds_select, baths_select, plot, data_table))

In [4]:
show(modify_doc)